In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import logging
import pprint
from os import listdir
from os.path import isfile, join

import pandas as pd
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_multiple_whitespaces, \
    remove_stopwords, stem_text
from gensim.corpora import MmCorpus, Dictionary
from gensim.models import TfidfModel, LdaModel, HdpModel, LsiModel

logging.basicConfig(format='\r%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

pp = pprint.PrettyPrinter(indent=4)
ENCODING = 'iso-8859-1'

d:\users\janvi\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def get_files(path):
    return [f for f in listdir(path) if isfile(join(path, f))]

## Analysis functions
### Text formating

In [3]:
ALLOWED_SHORT_WORDS = ['c', 'c#', 'r', '3d', '2d', '1d', '7z', 'qt']


def strip_short(word_list):    
    return [word for word in word_list if len(word) > 2 or word in ALLOWED_SHORT_WORDS]


def tokenize_text(text):
    processed = preprocess_string(text, filters=[strip_tags, strip_punctuation, 
                                                 strip_multiple_whitespaces, remove_stopwords, stem_text])
    return strip_short(processed)

### Data analysis

In [15]:
def get_real_tags(question_id):
    real_tags = tags_df[tags_df['Id'] == question_id]['Tag'].values
    return [tag for tag in real_tags]


def get_predicted_tags(question_index, model, dictionary):
    question_bow, question_id, text = get_bow_question(question_index, dictionary)
    return model[question_bow], question_id


def compare_real_predicted(question_index, model, dictionary, prob = 0.0):
    predicted_tags, question_id = get_predicted_tags(question_index, model, dictionary)
    predicted_tags = [p_tag for p_tag in predicted_tags if p_tag[1] > prob]
    real_tags = get_real_tags(question_id)
    correct_predictions = 0
    for tag in predicted_tags:
        if dictionary[tag[0]] in ' '.join(str(e) for e in real_tags):
            correct_predictions += 1            
    if len(real_tags) > len(predicted_tags):
        return correct_predictions, len(real_tags)    
    return correct_predictions, len(predicted_tags)


def get_bow_question(index, dictionary):
    tokenized_text = tokenize_text(csv_data['Title'][index])
    question_id = csv_data['Id'][index]
    text = csv_data['Title'][index]
    return dictionary.doc2bow(tokenized_text), question_id, text


def display_results(index, model, dictionary, prob=0.0):
    question_bow, question_id, text = get_bow_question(index, dictionary)
    predicted_tags = model[question_bow]
    tags = get_real_tags(question_id)
    print(f'Question {question_id}:\n{text}\nReal tags:\n{tags}\n\nPredicted tags:')
    [print(f'Tag: {dictionary[tag_id]}, Probability: {p}') for tag_id, p in predicted_tags if p >= prob]

In [16]:
def compare_n_tags(n, model, dictionary, prob=0.0):
    correct_predictions = 0
    all_predictions = 0
    for i in range(n):
        comparison = compare_real_predicted(i, model, dictionary, prob)
        correct_predictions += comparison[0]
        all_predictions += comparison[1]
    return correct_predictions, all_predictions

## Setup
### CSV files

In [6]:
PATH_TAGS = '../data/stacksample/Tags.csv'
tags_df = pd.read_csv(PATH_TAGS, encoding=ENCODING)

In [7]:
COLUMNS_QUESTION = ['Title', 'Body', 'Id']
QUESTIONS_PATH = '../data/stacksample/Questions.csv'
N_ROWS = 10000

csv_data  = pd.read_csv(QUESTIONS_PATH, encoding=ENCODING, usecols=COLUMNS_QUESTION, nrows=N_ROWS)
csv_data['All'] = csv_data['Title'].map(str) + csv_data['Body']

### Widgets

In [24]:
model_file_name = widgets.Dropdown(options=[f for f in get_files('../data/stacksample/models') if 'npy' not in f],
                                   description='Model file: ')
dict_file_name = widgets.Dropdown(options=get_files('../data/stacksample/dictionary'), description='Dictionary file: ')

text_index_slider1 = widgets.IntSlider(description='Text index: ', value=0, min=0, max=N_ROWS)
text_index_slider2 = widgets.IntSlider(description='Text index: ', value=0, min=0, max=N_ROWS)

probability_slider1 = widgets.FloatSlider(description='Tag probability: ', value=0.0, min=0.0, max=1.0, step=0.05)
probability_slider2 = widgets.FloatSlider(description='Tag probability: ', value=0.0, min=0.0, max=1.0, step=0.05)

analyse_btn = widgets.Button(description='Analyse model!', disabled=True)

def get_dictionary(file_name):
    return Dictionary.load(f'../data/stacksample/dictionary/{file_name}')

def get_model(file_name):
    return TfidfModel.load(f'../data/stacksample/models/{file_name}')
   
def test_example(b):
    display_results(text_index.value, model, corpus_dict)    

In [25]:
corpus_dict_widget = interactive(get_dictionary, file_name=dict_file_name)
model_widget = interactive(get_model, file_name=model_file_name)
display(corpus_dict_widget, model_widget)

In [27]:
interact_manual(display_results, index=text_index_slider1, p=probability_slider1,
                model=fixed(model_widget.result), dictionary=fixed(corpus_dict_widget.result))

<function __main__.display_results>

In [28]:
interact_manual(compare_n_tags, n=text_index_slider2, model=fixed(model_widget.result), 
                dictionary=fixed(corpus_dict_widget.result), prob=probability_slider2)

<function __main__.compare_n_tags>

In [19]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [20]:
CORPUS_FILE = 'titles_body_compact.mm'
corpus = MmCorpus(f'../data/stacksample/bow/{CORPUS_FILE}')

2018-06-05 01:57:10,770 : INFO : loaded corpus index from ../data/stacksample/bow/titles_body_compact.mm.index
2018-06-05 01:57:10,771 : INFO : initializing cython corpus reader from ../data/stacksample/bow/titles_body_compact.mm
2018-06-05 01:57:10,812 : INFO : accepted corpus with 1264216 documents, 2009852 features, 65879940 non-zero entries


In [24]:
pyLDAvis.gensim.prepare(model_widget.result, corpus, corpus_dict_widget.result)

d:\users\janvi\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


JoblibMemoryError: JoblibMemoryError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
d:\users\janvi\anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
d:\users\janvi\anaconda3\lib\runpy.

In [23]:
model_widget.result.show_topics()

[(11,
  '0.156*"android" + 0.051*"layout" + 0.032*"view" + 0.021*"activ" + 0.017*"r" + 0.017*"app" + 0.017*"intent" + 0.016*"com" + 0.015*"content" + 0.014*"parent"'),
 (29,
  '0.100*"valu" + 0.078*"row" + 0.073*"option" + 0.070*"select" + 0.030*"col" + 0.027*"echo" + 0.023*"php" + 0.015*"result" + 0.013*"var" + 0.009*"num"'),
 (20,
  '0.018*"type" + 0.014*"refer" + 0.013*"librari" + 0.012*"dll" + 0.012*"load" + 0.012*"the" + 0.011*"c" + 0.011*"object" + 0.011*"work" + 0.010*"method"'),
 (32,
  '0.060*"c" + 0.025*"build" + 0.023*"error" + 0.021*"command" + 0.021*"run" + 0.019*"compil" + 0.015*"file" + 0.014*"instal" + 0.011*"debug" + 0.010*"project"'),
 (40,
  '0.097*"kei" + 0.036*"properti" + 0.032*"entiti" + 0.024*"set" + 0.023*"valu" + 0.022*"titl" + 0.018*"book" + 0.015*"string" + 0.012*"type" + 0.010*"asset"'),
 (21,
  '0.055*"time" + 0.042*"2016" + 0.024*"textview" + 0.023*"dai" + 0.021*"start" + 0.017*"job" + 0.017*"year" + 0.017*"month" + 0.016*"date" + 0.013*"inflat"'),
 (27,


In [ ]:
def get_all_predicted_tags(model, dictionary):
    output_dict = {
        'Id': [],
        'Tags_p': []
    }
    for idx in range(len(csv_data)):
        prediction = get_predicted_tags(idx, model, dictionary)        
        tags = [(dictionary[tag_id], p) for tag_id, p in prediction[0]]
        output_dict['Tags_p'].append(tags)
        output_dict['Id'].append(prediction[1])
    return output_dict